## EP2 - MAC0460 - Introdução ao aprendizado de máquina
### Sabrina Araújo da Silva nUSP 12566182

Importações: utilizei as bibliotecas numpy e pandas, além da biblioteca de modelos Scikit-Learn que irá ser usada posteriormente. 

Utilizei classes específicas do Scikit-Learn, como o GridSearchCV para busca de parâmetros, o StandardScaler para pré-processamento de dados, e as classes para os modelos de Regressão Logística, SVM, Decision Tree e Random Forest.

In [215]:
import numpy as np 
import pandas as pd 

### Preparação do dataset

Para importar o dataset utilizei o arquivo "pokemon.csv", que contém os dados sobre os Pokémons.

Para a limpeza dos dados, as colunas que não são relevantes para a análise ou contêm muitos valores ausentes são excluídas do dataset. "type2", "abilities" e "classfication" foram retiradas como definido no enunciado do EP. Já as colunas "is_legendary", "percentage_male", "japanese_name", "name", "weight_kg", "height_m" foram retiradas por causa de conflitos no código, como valores iguais a 0, dados ausentes ou dados igual a NaN.


In [216]:

# Importação do dataset
arquivoDataset = "pokemon.csv"
data = pd.read_csv(arquivoDataset)

# Colunas não necessárias
colunasExcluir = ['type2', 'abilities', 'classfication', 'is_legendary', 'percentage_male', 'japanese_name', 'name', 'weight_kg', 'height_m']

# Colunas against_x que sejam diferentes de against_electric
colunasAgainstExcluir = [coluna for coluna in data.columns if coluna.startswith('against_') and coluna != 'against_electric']

# Retira colunas selecionadas
data = data.drop(columns=colunasExcluir + colunasAgainstExcluir)

# Filtra o dataset para manter apenas as linhas com type1 igual a "normal" ou "water"
data = data[data['type1'].isin(['normal', 'water'])]

X = data.drop(columns=['type1'])
y = data['type1'] 

print(X)
print(y)

     against_electric  attack  base_egg_steps  base_happiness  base_total  \
6                 2.0      48            5120              70         314   
7                 2.0      63            5120              70         405   
8                 2.0     103            5120              70         630   
15                2.0      45            3840              70         251   
16                2.0      60            3840              70         349   
..                ...     ...             ...             ...         ...   
772               1.0      95           30720               0         570   
774               1.0     115            5120              70         480   
778               2.0     105            3840              70         475   
779               0.5      60            5120              70         485   
787               2.0      75            3840              70         570   

    capture_rate  defense  experience_growth  hp  pokedex_number  sp_attack

A classe StandardScaler foi importada para realizar a padronização dos dados, garantindo que todas as variáveis tenham a mesma escala e evitando a influência de outliers. 

In [217]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X)
X_std = sc.transform(X)

print(X_std)


[[ 0.5703509  -0.88292165 -0.16884391 ... -0.13308681 -0.91779641
  -1.23603412]
 [ 0.5703509  -0.37736167 -0.16884391 ...  0.44621279 -0.33474454
  -1.23603412]
 [ 0.5703509   0.97079829 -0.16884391 ...  1.71343065  0.44265795
  -1.23603412]
 ...
 [ 0.5703509   1.03820628 -0.42562735 ...  0.08415054  0.98683969
   1.92378851]
 [-1.47730234 -0.47847366 -0.16884391 ...  0.84448126 -1.18988728
   1.92378851]
 [ 0.5703509   0.02708632 -0.42562735 ...  2.25652403  0.71474882
   1.92378851]]


O dataset foi dividido em duas partes: 75% foi alocado para o conjunto "atual", que contém os dados a serem utilizados para o treinamento atual, enquanto 25% foi destinado ao conjunto "final", que servirá para testar todos os modelos. O conjunto "atual" foi posteriormente subdividido em 70% para o conjunto de treinamento (X_train e y_train) e 30% para o conjunto de teste (X_test e y_test).

In [218]:
from sklearn.model_selection import train_test_split

# Divide o dataset em conjuntos de treinamento e teste
X_atual, X_final, y_atual, y_final = train_test_split(X_std, y, test_size=0.25, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_atual, y_atual, test_size=0.3, random_state=42)

#print(X_train)

### Treinamento

Agora, usamos os conjuntos X_train e y_train para treinar os seguintes modelos:

- Regressão Logística
- Support Vector Machine
- Decision Tree
- Random Forest

E posteriormente testamos os modelos com os conjuntos X_test e y_test.

#### Regressão Logística

O modelo de regressão logística foi importado da biblioteca Scikit-learn.


In [219]:
from sklearn.linear_model import LogisticRegression

LRmodel = LogisticRegression()


A seleção dos melhores parâmetros é realizada utilizando k-cross validation e grid search. Nesse processo, foi considerado o regularization parameter, que está indiretamente relacionado ao parâmetro C. O parâmetro C é o inverso do parâmetro de regularização, ou seja, quanto menor o valor de C, maior será a regularização aplicada ao modelo.

In [220]:
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# Procura os melhores parâmetros
kfold = model_selection.KFold(n_splits=10)
parameters = {'C': [0.1, 1, 10, 100]}
grid_search = GridSearchCV(LRmodel, parameters, scoring='accuracy', cv=kfold)
grid_search.fit(X_train, y_train)

print("Melhor parâmetro:", grid_search.best_params_)

# Treina o modelo com os melhores parâmetros
LRbest = grid_search.best_estimator_
LRbest.fit(X_train, y_train)

# Acurácia do modelo com os melhores parâmetros em relação aos dados de teste
accuracy = LRbest.score(X_test, y_test)
print("Acurácia do modelo RL: {:.2f}".format(accuracy))

Melhor parâmetro: {'C': 1}
Acurácia do modelo RL: 0.74


#### SVM - Support Vector Machine

Para usar o modelo SVM, usamos a classe SVC (Support Vector Classifier) do módulo svm do Scikit-learn. A classe SVC é utilizada para problemas de classificação com SVM.

In [221]:
from sklearn import svm

SVMmodel = svm.SVC()

A seleção dos melhores valores para os parâmetros C, kernel, gamma e grau do polinômio é realizada utilizando k-cross validation e grid search.

In [222]:
# Procura os melhores parâmetros
kfold = model_selection.KFold(n_splits=10)
parameters = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf', 'sigmoid'],
              'gamma': ['scale', 'auto', 0.1, 1], 'degree': [2, 3, 4]}
grid_search = GridSearchCV(SVMmodel, parameters, scoring='accuracy', cv=kfold)
grid_search.fit(X_train, y_train)

print("Melhores parâmetros:", grid_search.best_params_)

# Treina o modelo com os melhores parâmetros
SVMbest = grid_search.best_estimator_
SVMbest.fit(X_train, y_train)

# Acurácia do modelo em relação aos dados de teste
accuracy = SVMbest.score(X_test, y_test)
print("Acurácia do modelo SVM: {:.2f}".format(accuracy))

Melhores parâmetros: {'C': 1, 'degree': 2, 'gamma': 'scale', 'kernel': 'linear'}
Acurácia do modelo SVM: 0.72


#### Decision Tree

O modelo Decision Tree foi importado da biblioteca Scikit-learn.

In [223]:
from sklearn.tree import DecisionTreeClassifier

DTmodel = DecisionTreeClassifier()

A seleção dos melhores valores para os parâmetros "maximum depth of the tree" e "minimum number of samples required to be at a leaf node" é realizada utilizando k-cross validation e grid search.

In [224]:
# Procura os melhores parâmetros
kfold = model_selection.KFold(n_splits=10)
parameters = {'max_depth': [None, 5, 10, 20], 'min_samples_leaf': [1, 2, 5, 10]}
grid_search = GridSearchCV(DTmodel, parameters, scoring='accuracy', cv=kfold)
grid_search.fit(X_train, y_train)

# Treina o modelo com os melhores parâmetros
DTbest = grid_search.best_estimator_
DTbest.fit(X_train, y_train)

# Acurácia do modelo em relação aos dados de teste
accuracy = DTbest.score(X_test, y_test)
print("Acurácia do modelo DT: {:.2f}".format(accuracy))

Acurácia do modelo DT: 0.90


#### Random Forest

O modelo Random Forest foi importado da biblioteca Scikit-learn.

In [225]:
from sklearn.ensemble import RandomForestClassifier

RFmodel = RandomForestClassifier()

A seleção dos melhores valores para os parâmetros "number of estimators", "maximum depth of the tree" e "minimum number of samples required to be at a leaf node" é realizada utilizando k-cross validation e grid search.

In [226]:
# Procura os melhores parâmetros
kfold = model_selection.KFold(n_splits=10)
parameters = {'n_estimators': [100, 200, 500], 'max_depth': [None, 5, 10], 'min_samples_leaf': [1, 2, 5]}
grid_search = GridSearchCV(RFmodel, parameters, scoring='accuracy', cv=kfold)
grid_search.fit(X_train, y_train)

print("Melhores parâmetros:", grid_search.best_params_)

# Treina o modelo com os melhores parâmetros
RFbest = grid_search.best_estimator_
RFbest.fit(X_train, y_train)

# Acurácia do modelo em relação aos dados de teste
accuracy = RFbest.score(X_test, y_test)
print("Acurácia do modelo RF: {:.2f}".format(accuracy))

Melhores parâmetros: {'max_depth': 5, 'min_samples_leaf': 1, 'n_estimators': 500}
Acurácia do modelo RF: 0.84


Outros três modelos de classificação serão testados:

- Perceptron
- K-Nearest Neighbors (KNN)
- Naive Bayes

#### Perceptron

O modelo Perceptron foi importado da biblioteca Scikit-learn.

In [231]:
from sklearn.linear_model import Perceptron

Pmodel = Perceptron()

# Procura os melhores parâmetros
kfold = model_selection.KFold(n_splits=10)
parameters = {'alpha': [0.0001, 0.001, 0.01], 'max_iter': [100, 1000, 10000]}
grid_search = GridSearchCV(Pmodel, parameters, scoring='accuracy', cv=kfold)
grid_search.fit(X_train, y_train)

print("Melhores parâmetros:", grid_search.best_params_)

# Treina o modelo com os melhores parâmetros
Pbest = grid_search.best_estimator_
Pbest.fit(X_train, y_train)

# Acurácia do modelo em relação aos dados de teste
accuracy = Pbest.score(X_test, y_test)
print("Acurácia do modelo Perceptron: {:.2f}".format(accuracy))


Melhores parâmetros: {'alpha': 0.0001, 'max_iter': 100}
Acurácia do modelo Perceptron: 0.68


#### K-Nearest Neighbors (KNN)

O KNN se baseia nas instâncias mais próximas para classificar novas instâncias. O modelo K-Nearest Neighbors (KNN) foi importado da biblioteca Scikit-learn.

In [228]:
from sklearn.neighbors import KNeighborsClassifier

KNNmodel = KNeighborsClassifier()

# Procura os melhores valores para os parâmetros n_neighbors e weights
kfold = model_selection.KFold(n_splits=10)
parameters = {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']}
grid_search = GridSearchCV(KNNmodel, parameters, scoring='accuracy', cv=kfold)
grid_search.fit(X_train, y_train)

print("Melhores parâmetros:", grid_search.best_params_)

# Treina o modelo com os melhores parâmetros
KNNbest = grid_search.best_estimator_
KNNbest.fit(X_train, y_train)

# Acurácia do modelo em relação aos dados de teste
accuracy = KNNbest.score(X_test, y_test)
print("Acurácia do modelo KNN: {:.2f}".format(accuracy))


Melhores parâmetros: {'n_neighbors': 5, 'weights': 'uniform'}
Acurácia do modelo KNN: 0.72


#### Naive Bayes

O Naive Bayes assume independência condicional e estima probabilidades. O modelo Naive Bayes foi importado da classe GaussianNB da biblioteca Scikit-learn. Esse modelo não posssui ajuste de parâmetros.

In [229]:
from sklearn.naive_bayes import GaussianNB

NBmodel = GaussianNB()

# Treina o modelo
NBmodel.fit(X_train, y_train)

# Acurácia do modelo em relação aos dados de teste
accuracy = NBmodel.score(X_test, y_test)
print("Acurácia do modelo Naive Bayes: {:.2f}".format(accuracy))

Acurácia do modelo Naive Bayes: 0.66


### Escolha do melhor modelo

Agora será utilizado o conjunto de dados que foi separado previamente (X_final e y_final) para testar a acurácia de todos os modelos após o treinamento. Com base na acurácia final, é definido o melhor modelo.

In [232]:
# Imprime os valores de acurácia de todos os modelos
print('Acurácia final da Regressão Logística: {:.2f}'.format(LRbest.score(X_final, y_final)))
print('Acurácia final do SVM: {:.2f}'.format(SVMbest.score(X_final, y_final)))
print('Acurácia final da Decision Tree: {:.2f}'.format(DTbest.score(X_final, y_final)))
print('Acurácia final da Random Forest: {:.2f}'.format(RFbest.score(X_final, y_final)))
print()
print('Acurácia final do Perceptron: {:.2f}'.format(Pbest.score(X_final, y_final)))
print('Acurácia final do K-Nearest Neighbors: {:.2f}'.format(KNNbest.score(X_final, y_final)))
print('Acurácia final do Naive Bayes: {:.2f}'.format(NBmodel.score(X_final, y_final)))
print()

# Identifica o melhor modelo com base na maior acurácia
melhor_modelo = max([LRbest.score(X_final, y_final), SVMbest.score(X_final, y_final), DTbest.score(X_final, y_final), RFbest.score(X_final, y_final)])
if melhor_modelo == LRbest.score(X_final, y_final):
    print("O melhor modelo é a Regressão Logística.")
elif melhor_modelo == SVMbest.score(X_final, y_final):
    print("O melhor modelo é o SVM.")
elif melhor_modelo == DTbest.score(X_final, y_final):
    print("O melhor modelo é a Decision Tree.")
elif melhor_modelo == RFbest.score(X_final, y_final):
    print("O melhor modelo é o Random Forest.")
elif melhor_modelo == Pbest.score(X_final, y_final):
    print("O melhor modelo é o Naive Bayes.")
elif melhor_modelo == KNNbest.score(X_final, y_final):
    print("O melhor modelo é o K-Nearest Neighbors.")
elif melhor_modelo == NBmodel.score(X_final, y_final):
    print("O melhor modelo é o Naive Bayes.")


Acurácia final da Regressão Logística: 0.80
Acurácia final do SVM: 0.80
Acurácia final da Decision Tree: 0.85
Acurácia final da Random Forest: 0.87

Acurácia final do Perceptron: 0.60
Acurácia final do K-Nearest Neighbors: 0.75
Acurácia final do Naive Bayes: 0.75

O melhor modelo é o Random Forest.


#### Matrizes de confusão

Com base no conjunto de dados para o teste final, os resultados foram representados abaixo em matrizes de confusão:

In [213]:
from sklearn.metrics import confusion_matrix

# Função para calcular e imprimir a matriz de confusão
def confusionMatrix(model, X, y):
    y_pred = model.predict(X)
    confusion = confusion_matrix(y, y_pred)
    print(confusion)

# Matriz de confusão para Regressão Logística
print("Matriz de confusão para Regressão Logística:")
confusionMatrix(LRbest, X_final, y_final)
print()

# Matriz de confusão para SVM
print("Matriz de confusão para SVM:")
confusionMatrix(SVMbest, X_final, y_final)
print()

# Matriz de confusão para Decision Tree
print("Matriz de confusão para Decision Tree:")
confusionMatrix(DTbest, X_final, y_final)
print()

# Matriz de confusão para Random Forest
print("Matriz de confusão para Random Forest:")
confusionMatrix(RFbest, X_final, y_final)
print()

# Matriz de confusão para Perceptron
print("Matriz de confusão para Perceptron:")
confusionMatrix(Pbest, X_final, y_final)
print()

# Matriz de confusão para KNN
print("Matriz de confusão para K-Nearest Neighbors:")
confusionMatrix(KNNbest, X_final, y_final)
print()

# Matriz de confusão para Naive Bayes
print("Matriz de confusão para Naive Bayes:")
confusionMatrix(NBmodel, X_final, y_final)


Matriz de confusão para Regressão Logística:
[[23  7]
 [ 4 21]]

Matriz de confusão para SVM:
[[23  7]
 [ 4 21]]

Matriz de confusão para Decision Tree:
[[20 10]
 [ 3 22]]

Matriz de confusão para Random Forest:
[[24  6]
 [ 1 24]]

Matriz de confusão para Perceptron:
[[15 15]
 [ 7 18]]

Matriz de confusão para K-Nearest Neighbors:
[[20 10]
 [ 4 21]]

Matriz de confusão para Naive Bayes:
[[22  8]
 [ 6 19]]


#### Importância das features

Uso da Random Forest para obter uma estimativa de quais features foram mais importantes para a classificação dos pokémons.

In [102]:
# Obtém as importâncias das features usando o modelo Random Forest
importances = RFbest.feature_importances_
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feature_importances = feature_importances.sort_values('Importance', ascending=False)

# Imprime as importâncias das features
print("Importância das Features:")
print(feature_importances)


Importância das Features:
              Feature  Importance
0    against_electric    0.243598
10          sp_attack    0.109782
4          base_total    0.076333
2      base_egg_steps    0.074573
1              attack    0.073187
12              speed    0.065950
11         sp_defense    0.064077
9      pokedex_number    0.063630
6             defense    0.063455
8                  hp    0.054710
7   experience_growth    0.038081
5        capture_rate    0.034194
13         generation    0.030888
3      base_happiness    0.007543
